In [241]:
import pandas as pd
import numpy as np

## Load in the 2016 votes by county

In [242]:
vote_df = pd.read_csv('../data/2016-vote-fips-acp.csv', dtype={"fips":object})

In [243]:
vote_df.head()

,fips,acp_type,county,st,total_votes,lead,st_name,trump,clinton,other
0,01001,1,Autauga County,AL,24759,Donald Trump,Alabama,18172,5936,651
1,01051,1,Elmore County,AL,36927,Donald Trump,Alabama,27634,8443,850
2,01083,1,Limestone County,AL,39792,Donald Trump,Alabama,29067,9468,1257
3,01115,1,St. Clair County,AL,38054,Donald Trump,Alabama,31651,5589,814
4,01117,1,Shelby County,AL,99442,Donald Trump,Alabama,73020,22977,3445


## Do vote experimetation here

In [259]:
# No change -- same as 2016
vote_df['rep'] = vote_df['trump']
vote_df['dem'] = vote_df['clinton']

## Electoral College Calculation

In [260]:
vote_by_state = pd.pivot_table(vote_df, index='st_name', values=['rep','dem'], aggfunc=np.sum)

In [261]:
vote_by_state.reset_index(inplace=True)

### Calculate Nebraska & Maine's Districts

Nebraska and Maine give two electoral votes to whomever wins the statewide vote and then gives one additional electoral vote for the winner of each of their congressional districts.

Nebraska's congressional districts follow county boundaries, and Maine almost does (I'm putting all of Kennebec County in Maine-1).

Here we marry the county counts to the FIPS codes of the counties in each of the congressional districts.

In [262]:
me_ne_df = pd.read_csv('../data/maine-nebraska-cds.csv', dtype={'fips':object})

In [263]:
me_ne_votes = me_ne_df.merge(vote_df, on="fips", how="left")

In [264]:
me_ne_districts = pd.pivot_table(me_ne_votes, index='cong_dist', values=['rep','dem'], aggfunc=np.sum)

In [265]:
me_ne_districts.reset_index(inplace=True)

In [266]:
me_ne_districts.rename(columns={"cong_dist":"st_name"}, inplace=True)

In [267]:
electoral_df = pd.read_csv('../data/electoral_votes.csv')

In [268]:
# join this to the state list
vote_with_districts = pd.concat([vote_by_state, me_ne_districts])

In [269]:
vote_electoral = vote_with_districts.merge(electoral_df, left_on="st_name", right_on="state", how="left" )

In [270]:
vote_electoral.loc[vote_electoral['dem'] > vote_electoral['rep'], 'd_votes'] = vote_electoral['electoral_votes']
vote_electoral.loc[vote_electoral['rep'] > vote_electoral['dem'], 'r_votes'] = vote_electoral['electoral_votes']


In [271]:
vote_electoral

,st_name,dem,rep,state,electoral_votes,d_votes,r_votes
0,Alabama,744137.94,1331437.55,Alabama,9,NaN,9.0
1,Alaska,118504.62,164980.47,Alaska,3,NaN,3.0
2,Arizona,1184390.34,1264925.01,Arizona,11,NaN,11.0
3,Arkansas,388103.88,691720.72,Arkansas,6,NaN,6.0
4,California,8928863.76,4528648.10,California,55,55.0,NaN
5,Colorado,1365647.40,1214508.84,Colorado,9,9.0,NaN
6,Connecticut,915523.44,679947.15,Connecticut,7,7.0,NaN
7,Delaware,240315.06,186978.27,Delaware,3,3.0,NaN
8,District of Columbia,288486.60,12850.23,District of Columbia,3,3.0,NaN
9,Florida,4595074.50,4664064.86,Florida,29,NaN,29.0


In [272]:
print(f"Democrat:   {np.sum(vote_electoral['d_votes'])}")
print(f"Republican: {np.sum(vote_electoral['r_votes'])}")
if (np.sum(vote_electoral['d_votes']) > np.sum(vote_electoral['r_votes'])):
        print ("Democrat wins")
elif (np.sum(vote_electoral['r_votes']) > np.sum(vote_electoral['d_votes'])):
        print ("Republican wins")
else:
        print ("Electoral college tie")
           

Democrat:   248.0
Republican: 290.0
Republican wins


In [273]:
print(np.sum(vote_electoral['electoral_votes']))

538
